# Segmenting and Clustering Neighborhoods in Toronto

Required to explore and cluster the neighborhoods in Toronto.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
import geocoder # import geocoder
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, features='html')
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string.rstrip());      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1].rstrip())
                else:
                    neighborhoods.append(cell.string.rstrip()); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors_raw = pd.DataFrame(columns=column_names)

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors_raw = neighbors_raw.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

In [5]:
# neighbors_raw.groupby('PostalCode')[['Borough','Neighborhood']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
# neighbors_raw.groupby('PostalCode')[['Borough','Neighborhood']].apply(lambda g: g.values.tolist()).to_dict()

# find unique value of Neighborhood
tmp_Borough=neighbors_raw.groupby('PostalCode')['Borough'].unique().apply(list).to_dict()
# finding and grouping values of Neighborhood for the same PostaCode
tmp_Neighborhood=neighbors_raw.groupby('PostalCode')['Neighborhood'].apply(list).to_dict()

t1=pd.DataFrame.from_dict(list(tmp_Borough.items()))
t1.columns=column_names[0:2]
t2=pd.DataFrame.from_dict(list(tmp_Neighborhood.items()))
t2.columns=[column_names[0],column_names[2]]
t1['Neighborhood']=t2['Neighborhood']

#ss.applymap(lambda x: x[0] if isinstance(x, list) else x)

# creating new dataframe 
neighbors = pd.DataFrame(columns=column_names)

neighbors['PostalCode']=t1['PostalCode']
neighbors['Borough']=t1['Borough'].str.get(0)
neighbors["Neighborhood"]=t1["Neighborhood"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN
...,...,...,...,...,...
98,M9N,York,Weston,NaN,NaN
99,M9P,Etobicoke,Westmount,NaN,NaN
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",NaN,NaN
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",NaN,NaN


Getting LAT and LON

In [6]:
# initialize your variable to None
lat_lng_coords = None

for k in range(0, len(neighbors)-1):
    print('{}/{} - {}'.format(k,len(neighbors)-1,neighbors['PostalCode'][k]))
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(neighbors['PostalCode'][k]))
    lat_lng_coords = g.latlng

    neighbors['Latitude'][k] = lat_lng_coords[0]
    neighbors['Longitude'][k] = lat_lng_coords[1]
    

0/102 - M1B
1/102 - M1C
2/102 - M1E
3/102 - M1G
4/102 - M1H
5/102 - M1J
6/102 - M1K
7/102 - M1L
8/102 - M1M
9/102 - M1N
10/102 - M1P
11/102 - M1R
12/102 - M1S
13/102 - M1T
14/102 - M1V
15/102 - M1W
16/102 - M1X
17/102 - M2H
18/102 - M2J
19/102 - M2K
20/102 - M2L
21/102 - M2M
22/102 - M2N
23/102 - M2P
24/102 - M2R
25/102 - M3A
26/102 - M3B
27/102 - M3C
28/102 - M3H
29/102 - M3J
30/102 - M3K
31/102 - M3L
32/102 - M3M
33/102 - M3N
34/102 - M4A
35/102 - M4B
36/102 - M4C
37/102 - M4E
38/102 - M4G
39/102 - M4H
40/102 - M4J
41/102 - M4K
42/102 - M4L
43/102 - M4M
44/102 - M4N
45/102 - M4P
46/102 - M4R
47/102 - M4S
48/102 - M4T
49/102 - M4V
50/102 - M4W
51/102 - M4X
52/102 - M4Y
53/102 - M5A
54/102 - M5B
55/102 - M5C
56/102 - M5E
57/102 - M5G
58/102 - M5H
59/102 - M5J
60/102 - M5K
61/102 - M5L
62/102 - M5M
63/102 - M5N
64/102 - M5P
65/102 - M5R
66/102 - M5S
67/102 - M5T
68/102 - M5V
69/102 - M5W
70/102 - M5X
71/102 - M6A
72/102 - M6B
73/102 - M6C
74/102 - M6E
75/102 - M6G
76/102 - M6H
77/102 - 

In [7]:
neighbors.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8115,-79.1955
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7857,-79.1587
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7658,-79.1752
3,M1G,Scarborough,Woburn,43.7684,-79.2176
4,M1H,Scarborough,Cedarbrae,43.7697,-79.2394


In [8]:
neighbors.shape

(103, 5)